In [40]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-final-schedule", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3343,application_1618324153128_2983,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3065,application_1618324153128_2643,pyspark,error,Link,Link,None,
3330,application_1618324153128_2969,pyspark,idle,Link,Link,None,
3336,application_1618324153128_2976,pyspark,idle,Link,Link,None,
3342,application_1618324153128_2982,pyspark,error,Link,Link,None,
3343,application_1618324153128_2983,pyspark,idle,Link,Link,None,✔


In [41]:
%%send_to_spark -i username -t str -n username

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [42]:
print('We are using Spark %s' % spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are using Spark 2.3.2.3.1.4.0-315

In [104]:
stops = spark.read.orc('/data/sbb/orc/geostops')
stop_times = spark.read.csv("/data/sbb/csv/timetable/stop_times/2019/05/07/stop_times.csv", header=True)
routes = spark.read.csv('/data/sbb/csv/timetable/routes/2019/05/07/routes.csv', header=True )
calendar = spark.read.csv('/data/sbb/csv/timetable/calendar/2019/05/07/calendar.csv', header=True)
trips = spark.read.csv('/data/sbb/csv/timetable/trips/2019/05/07/trips.csv', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
from pyspark.sql.functions import acos, asin, cos, sin, lit, toRadians, sqrt

def haversine(theta):
    return (lit(1) - cos(theta)) / lit(2)

def haversine_dist(latitude_x, longitude_x, latitude_y, longitude_y):
    latitude_x, longitude_x, latitude_y, longitude_y = toRadians(latitude_x), toRadians(longitude_x),\
                                                       toRadians(latitude_y), toRadians(longitude_y)
    h = haversine(latitude_x - latitude_y) + cos(latitude_x) * cos(latitude_y) * haversine(longitude_x - longitude_y)
    earth_radius = 6371.0
    return acos(lit(1) - lit(2) * h) * earth_radius

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
stops.withColumn('distance_zurich_HB' , haversine_dist(lit(47.378177), lit(8.540192), stops.stop_lat, stops.stop_lon)).show(n=5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+----------------+----------------+-------------+--------------+------------------+
|    stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|distance_zurich_HB|
+-----------+--------------------+----------------+----------------+-------------+--------------+------------------+
|    8711790|      Bouilly Mairie|48.1917218939628|3.99698199202347|         null|              | 351.2269447014103|
|    8573112|              Gwüest|46.6525768281602|8.51647805228063|         null|              | 80.70308424840454|
|8014471:0:3|            Albbruck|47.5923598499222|8.13177453180289|         null|      8014471P| 38.84665273235225|
|    8774562|          Perrignier| 46.303453968822|6.42497549173821|         null|              | 200.3995158929776|
|    8592544|Schaffhausen, Gre...|47.6988945057898|8.64765005008602|         null|              |36.563071544639875|
+-----------+--------------------+----------------+-------------